In [ ]:
%matplotlib inline
import os 
import sys
import numpy as np
from numpy import mean, std
import nibabel as nb 
import pandas as pd
import seaborn as sns#
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, ttest_ind, mannwhitneyu, wilcoxon, spearmanr,pearsonr, fisher_exact,kstest
from scipy.stats import ks_2samp, kstest, anderson, anderson_ksamp, levene
from statsmodels.sandbox.stats.multicomp import fdrcorrection0
from matplotlib import colors
from sklearn import preprocessing
import statsmodels.api as sm
pd.options.display.max_columns = 999
import statsmodels.formula.api as smf
import scipy.stats as ss 
import math 
from outliers import smirnov_grubbs as grubbs
from matplotlib import rc, font_manager

plt.rcParams['font.family'] = 'Myriad Pro'
sns.set_style('white')

datadir       = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSM/FEB17'
mrsdir        = '/Users/kanaan/SCR/Dataframes_20150820/REVISION_DF'
clinical_dir  = '/Users/kanaan/SCR/Dataframes_20150820/DF_clinical/'
save_fig_dir  = '/Users/kanaan/Google Drive/TS-EUROTRAIN/Papers/2016_QSM_paper/Figures_python'

measures = ['Puta', 'Caud', 'Thal','SN', 'STN', 'RN',  'GPi' , 'GPe']

drop_c = ['WSKT','SC1T','LL5T', 'GSAT'] 
# WSKT,SC1T,LL5T Young Females scanned extra for MRS review. 
# GSAT removed since HSPP female removed 
drop_p = ['AA8P', 'RA9P', 'STDP','NL2P','HSPP', 'DF2P'] 
# AA8P motion outliers via mehalonobis distance for Quality metrics 
# rest – mehalonobis distance for SN,STN,Puta


In [ ]:
def get_all_dfs(img, population, study_id, trt=False):
    quant = 'median'
    QSM = pd.read_csv(os.path.join(datadir, '%s_%s_%s_%s.csv'%(img, quant, population, study_id)),index_col = 0)
    for i in measures:
        QSM[i] = QSM[i] * 1000
    
    QSM['Gender']  = QSM['Gender'].map({' F': 1, ' M': 0})
    
    CLN = pd.read_csv(os.path.join(clinical_dir, 'clinical_%s_a.csv'%(population)), index_col=0)
    CLN.columns = ['CLN_' + str(col) for col in CLN.columns]    

    def get_mrs(vox1,vox2,vox3):
        if trt == False:
            df1 = pd.read_csv(os.path.join(mrsdir, 'moco_df_%s_%s_%s.csv'%(vox1, population, study_id)),index_col = 0)
            df2 = pd.read_csv(os.path.join(mrsdir, 'moco_df_%s_%s_%s.csv'%(vox2, population, study_id)),index_col = 0)
            df3 = pd.read_csv(os.path.join(mrsdir, 'moco_df_%s_%s_%s.csv'%(vox3, population, study_id)),index_col = 0)
        elif trt == True:
            df1 = pd.read_csv(os.path.join(mrsdir, 'moco_df_%s_%s_trt_%s.csv'%(vox1, population, study_id)),index_col = 0)
        df1.columns = ['%s_'%vox1 + str(col) for col in df1.columns]
        df2.columns = ['%s_'%vox2 + str(col) for col in df2.columns]
        df3.columns = ['%s_'%vox3 + str(col) for col in df3.columns]
        
        return df1,df2,df3
    
    xstr,xtha,xacc = get_mrs('STR', 'THA', 'ACC')
    df = pd.concat([QSM, xstr,xtha,xacc, CLN], axis=1, join_axes=[QSM.index])
    df['L_Caud_Puta']    = (df['L_Puta'] + df['L_Caud']) / 2
    df['R_Caud_Puta']    = (df['R_Puta'] + df['R_Caud']) / 2
    df['Caud_Puta']      = (df['L_Caud_Puta'] + df['R_Caud_Puta']) / 2
    df['L_BrainStem']    = (df['L_STN'] + df['L_SN'] +  df['L_RN']) / 3
    df['R_BrainStem']    = (df['R_STN'] + df['R_SN'] +  df['R_RN']) / 3
    df['L_BasalGanglia'] = (df['L_Puta'] + df['L_Caud'] +  df['L_Pall'] ) / 3
    df['R_BasalGanglia'] = (df['R_Puta'] + df['R_Caud'] +  df['R_Pall'] ) / 3
    df['L_ALL_NUCLEI'] = (df['L_BrainStem'] + df['L_BasalGanglia'] + df['L_DN']) / 3
    df['R_ALL_NUCLEI'] = (df['R_BrainStem'] + df['R_BasalGanglia'] + df['R_DN']) / 3
    df['ALL_NUCLEI'] = (df['BrainStem'] + df['BasalGanglia']) /2
    
    
    for i in ['ALL_NUCLEI', 'L_ALL_NUCLEI', 'R_ALL_NUCLEI', 'Caud_Puta', 'MRS_STR', 'MRS_THA', 'MRS_ACC',
              'BasalGanglia', 'R_BasalGanglia', 'L_BasalGanglia']:
        df[i] = df[i] * 1000
    
    return df

controls_a = get_all_dfs('QSM', 'controls', 'a').drop(drop_c, axis = 0) 
patients_a = get_all_dfs('QSM', 'patients', 'a').drop(drop_p, axis = 0)
#controls_b = get_all_dfs('QSM', 'controls', 'b', trt=True)#.drop(drop_c, axis = 0) 
#patients_b = get_all_dfs('QSM', 'patients', 'b', trt=True)#.drop(drop_p, axis = 0) 

controls_a['Population'] = 0
patients_a['Population'] = 1
    
#controls_b.index = controls_b.index + 'b'
#patients_b.index = patients_b.index + 'b'

DF_ALL =pd.concat([controls_a, patients_a], axis=0)


In [ ]:
def grubbs_outliers(df, measure, alpha = 0.15):
    from outliers import smirnov_grubbs as grubbs
    return [str(i) for i in df.index if i not in grubbs.test(df[measure], alpha).index] 
def grubbs_outliersx(df, alpha = 0.15):
    from outliers import smirnov_grubbs as grubbs
    return [str(i) for i in df.index if i not in grubbs.test(df, alpha).index] 

def gESD_outliers(df, measure, maxOLs =10, alpha = 0.05):
    from PyAstronomy import pyasl
    index = pyasl.generalizedESD(df[measure],maxOLs = maxOLs, alpha =alpha, fullOutput=True)[1]
    return [df.index[i] for i in index] 

def gESD_outliersx(df, maxOLs =10, alpha = 0.05):
    from PyAstronomy import pyasl
    index = pyasl.generalizedESD(df,maxOLs = maxOLs, alpha =alpha, fullOutput=True)[1]
    return [df.index[i] for i in index] 

def sortdf(df,m):
    return df.sort(m)[m]
 
def plot_partial_correlation(data, x, y,xlim = None, ylim = None, regressors = None, color = None, annotate = 0, fig_name = None, 
                             dpi = 100, labels = True, jitter = None,fit_reg=True, scatter=1,
                             plot_marg = 1, ci = 95, lstyle='-',lw=2.5):
    
    regressors_columns = [data[regressor] for regressor in regressors]
    df_reg = pd.concat(regressors_columns, axis = 1).dropna(axis=0) 
    df = pd.concat([df_reg, data[x], data[y]], axis=1).dropna()
    
    # inititlize grid and plot regression and margins 
    grid = sns.JointGrid(x, y, data=df,xlim=xlim, ylim=ylim)#, size=size, ratio=ratio, space=space, #) 
    
    if plot_marg:
        grid.plot_marginals(sns.distplot, color = color, hist=False, kde = True, rug = 0, kde_kws={'shade':True})  
    
    grid.plot_joint(sns.regplot, fit_reg=fit_reg, scatter= scatter, color = color, x_partial = df[regressors], y_jitter = jitter, 
                    ci = ci, line_kws={'linestyle': lstyle, 'lw':lw}, scatter_kws={"s": 25})
    if labels:
        grid.set_axis_labels(x, y, fontsize= 15, weight='bold', color='blue', labelpad=10)
    else:
        grid.set_axis_labels(None,None)
    
    ## calculate regression parameters 
    
    formula = [ '%s ~ %s'%(y,x) + ' + %s'%regressor for regressor in regressors][0]
    model   = smf.ols(formula=formula, data= df)
    result = model.fit() 
    p_val  = np.round(result.pvalues[1], 7)
    pcor   = math.copysign(np.round(np.sqrt(result.rsquared), 3) , result.params[1])
    plt.ylabel(y)
    plt.xlabel(x)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    
    if annotate == 1:
        annotate = (plt.xlim()[1]*0.6,  plt.ylim()[1]*0.95)
        plt.annotate('R=%s, P=%s' %(pcor, p_val), xy = annotate, fontsize = 13,  color='r')
    elif annotate == 0:
        pass
    else:
        plt.annotate('R=%s, P=%s' %(pcor, p_val), xy = annotate, fontsize = 13,  color='r')
    
    
    if fig_name:
        plt.savefig(os.path.join(save_fig_dir, fig_name), dpi = dpi, bbox_inches='tight', transparent = True)
    

    print pcor, p_val
    return result

def plot_joint(df,m1,m2, drop = []):
    DF  = pd.DataFrame([df[m1] , df[m2]] ).T.dropna()
    sns.jointplot(x=m1, y=m2, data=DF.drop(drop, axis = 0), kind = 'reg')
    

In [ ]:
def get_resid(df,measure, regressors):
        formula = [ '%s ~ %s'%(measure,regressors[0])  + ' + %s'%regressor for regressor in regressors[1:]][0]#  
        res= smf.ols(formula=formula, data= df).fit().resid
        return res #+ np.mean(df[measure].dropna())
    
    
def plt_resid(data, x, y, regressors = None, color = None, annotate = None, fig_name = None, 
              dpi = 100, labels = True, jitter = None,fit_reg=True, scatter=1,
              plot_marg = 1, xlim = None, ylim = None):
    
    x_resid = get_resid(data,x, regressorsx)
    y_resid = get_resid(data,y, regressorsx)
    print 'Grubbs Outliers', grubbs_outliersx(x_resid), grubbs_outliersx(y_resid)
    print 'gESD Outliers',gESD_outliersx(x_resid),gESD_outliersx(y_resid)
    df = pd.concat([x_resid, y_resid], axis=1).dropna()
    df.columns = [x,y]
    
    grid = sns.JointGrid(x, y, data=df,xlim=xlim, ylim=ylim)#, size=size, ratio=ratio, space=space, #) 
    
    if plot_marg:
        grid.plot_marginals(sns.distplot, color = color, hist=False, kde = True, rug = 0, kde_kws={'shade':True})  
    
    grid.plot_joint(sns.regplot, fit_reg=fit_reg, scatter= scatter, color = color, y_jitter = jitter)
    if labels:
        grid.set_axis_labels(x, y, fontsize= 15, weight='bold', color='blue', labelpad=10)
    else:
        grid.set_axis_labels(None,None)
    formula = [ '%s ~ %s'%(y,x)][0]
    model   = smf.ols(formula=formula, data= df)
    result = model.fit() 
    p_val  = np.round(result.pvalues[1], 8)
    pcor   = math.copysign(np.round(np.sqrt(result.rsquared), 3) , result.params[1])
    plt.ylabel(y)
    plt.xlabel(x)
    if annotate:
        plt.annotate('R=%s, P=%s' %(pcor, p_val), xy = annotate, fontsize = 13,  color='r')

    if fig_name:
        plt.savefig(os.path.join(save_fig_dir, fig_name), dpi = dpi, bbox_inches='tight', transparent = True)



# Glutamate STR

In [ ]:
drp = ['KDDP','YU1P','TT3P']
drc = []
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'STR_Gln_Glu'
y = 'MRS_STR'

regressorsx = ['Age', 'Gender', 'QI1_MAG', 'EFC_MAG','STR_FWHM']
xlim  = (0.1, 0.8)
ylim  = (-30,40)
dpi = 800 

print gESD_outliers(patients_a, x, alpha = .05)
print gESD_outliers(controls_a, x, alpha = .05)
print gESD_outliers(patients_a, y, alpha = .05)
print gESD_outliers(controls_a, y, alpha = .05)

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (0.5,0.045),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         #fig_name = 'CORR_MRSglnglu_QSM_STR_reg_controls.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7, #annotate = (0.5,0.04),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         #fig_name = 'CORR_MRSglnglu_QSM_STR_reg_patients.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,  #annotate = (0.5,0.035),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         #fig_name = 'CORR_MRSglnglu_QSM_STR_reg_ALL.png'
                        )

# Glutamate THALAMUS

In [ ]:
drp = []#
drc = [ 'RMNT']
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'THA_Gln_Glu'
y = 'Thal'

regressorsx = ['Age', 'Gender', 'QI1_MAG', 'EFC_MAG']
xlim  = (0, 1.2)
ylim  = (-100,50)
dpi = 800 

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (0.65,0.13),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'CORR_MRSglnglu_QSM_THA_c.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7,  #annotate = (0.65,0.12),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_MRSglnglu_QSM_THA_p.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,   #annotate = (0.65,0.11),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_MRSglnglu_QSM_THA_all.png'
                        )

# ACC

In [ ]:
drp = []#
drc = []
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'ACC_Gln_Glu'
y = 'MRS_ACC'

regressorsx = ['Age', 'Gender', 'QI1_MAG']
xlim  = (0, 0.5)
ylim  = (-105,50)
dpi = 800 

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (0.65,0.13),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'CORR_MRSglnglu_QSM_acc_c.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7,  #annotate = (0.4,0.12),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_MRSglnglu_QSM_acc_p.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,   #annotate = (0.65,0.11),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_MRSglnglu_QSM_acc_all.png'
                        )

# FWHM

In [ ]:

###########

#PLOTS STD of QSM in MRSvoxel vs MRS_FWHM


sns.jointplot(x = 'STR_FWHM', y = 'MRS_STR', kind='reg', data = DF_ALL)#.drop(['KDDP','YU1P','TT3P'], axis = 0),)


In [ ]:
sns.jointplot(x = 'ACC_FWHM', y = 'MRS_ACC', data = DF_ALL['KDDP','YU1P','TT3P'], kind='reg')


In [ ]:
sns.jointplot(x = 'THA_FWHM', y = 'MRS_THA', data = DF_ALL, kind='reg')


# Relationships with Ferritin

In [ ]:
print grubbs_outliers(patients_a, 'CLN_Ferritin', alpha = .05)
print grubbs_outliers(controls_a, 'CLN_Ferritin', alpha = .05)
print grubbs_outliers(patients_a, 'BrainStem', alpha = .05)
print grubbs_outliers(controls_a, 'BrainStem', alpha = .05)
print grubbs_outliers(patients_a, 'BasalGanglia', alpha = .05)
print grubbs_outliers(controls_a, 'BasalGanglia', alpha = .05)

In [1]:
drp = ['SA5U']#
drc = ['PU2T', 'SMVX']
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'CLN_Ferritin'
y = 'ALL_NUCLEI'

regressorsx = ['Age', 'Gender', 'QI1_MAG', 'EFC_MAG']
xlim  = (-400, 800)
ylim  = (0,80)
dpi = 800 

print gESD_outliers(patients_a, x, alpha = .05)
print gESD_outliers(controls_a, x, alpha = .05)
print gESD_outliers(patients_a, y, alpha = .05)
print gESD_outliers(controls_a, y, alpha = .05)

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (350,0.075),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'FERR_CORR_QSM_ALL_reg_controls.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7, #annotate =  (350,0.07),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'FERR_CORR_QSM_ALL_reg_patients.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,  #annotate = (350,0.065),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'FERR_CORR_QSM_ALL_reg_ALL.png'
                        )

NameError: name 'DF_ALL' is not defined

In [ ]:
drp = ['SA5U']#
drc = ['BH5T', 'GSNT']#'PU2T', 'SMVX', 
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'CLN_Ferritin'
y = 'L_BasalGanglia'

regressorsx = ['Age', 'Gender', 'QI1_MAG', 'EFC_MAG']
xlim  = (-200, 600)
ylim  = (-10,70)
dpi = 500 

print gESD_outliers(patients_a, x, alpha = .05)
print gESD_outliers(controls_a, x, alpha = .05)
print gESD_outliers(patients_a, y, alpha = .05)
print gESD_outliers(controls_a, y, alpha = .05)

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (300,0.055),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'FERR_CORR_QSM_BG_reg_controls.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7,  #annotate =  (300,0.05),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                        fig_name = 'FERR_CORR_QSM_BG_reg_patients.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,  #annotate = (300,0.045),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'FERR_CORR_QSM_BG_reg_ALL.png'
                        )

In [ ]:
drp = ['SA5U']#
drc = ['PU2T', 'SMVX', 'BH5T']
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'CLN_Ferritin'
y = 'BrainStem'

regressorsx = ['Age', 'Gender', 'QI1_MAG', 'EFC_MAG']
xlim  = (-200, 700)
ylim  = (.04,0.18)
dpi = 500 

print gESD_outliers(patients_a, x, alpha = .05)
print gESD_outliers(controls_a, x, alpha = .05)
print gESD_outliers(patients_a, y, alpha = .05)
print gESD_outliers(controls_a, y, alpha = .05)

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (300,0.165),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'FERR_CORR_QSM_BSt_reg_controls.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', ci = None,
                         labels = False,lstyle='--', lw=0.7, #annotate =  (300,0.16),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'FERR_CORR_QSM_BSt_reg_patients.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,  #annotate = (300,0.155),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'FERR_CORR_QSM_BSt_reg_ALL.png'
                        )

# relationship with AGE

In [ ]:
drp = ['SA5U']#
drc = []
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'CLN_Age'
y = 'CLN_Ferritin'

regressorsx = ['Gender']
xlim  = (0, 70)
ylim  = (-200,600)
dpi = 500 

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', #ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (35,500),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'CORR_Ferr_age_c.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', #ci = None,
                         labels = False,lstyle='--', lw=0.7,  #annotate = (35,470),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_Ferr_age_p.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,   #annotate = (35,440),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_Ferr_age_all.png'
                        )

In [ ]:
drp = ['SA5U']#
drc = []
df = DF_ALL.drop(drp+drc ,axis=0)
x = 'CLN_Age'
y = 'ALL_NUCLEI' 

regressorsx = ['Gender', 'QI1_MAG', 'EFC_MAG']
xlim  = (0, 70)
ylim  = (10,110)
dpi = 500 

plot_partial_correlation(controls_a.drop(drc,axis=0), x, y, regressors = regressorsx, color = 'b', #ci = None,
                         labels = False, lstyle='--', lw=0.7, #annotate = (35,0.07),
                         fit_reg=1, scatter=1, plot_marg = 1, dpi = dpi, xlim=xlim,ylim=ylim, 
                         fig_name = 'CORR_Age_ChiAll_c.png'
                        )
plot_partial_correlation(patients_a.drop(drp,axis=0), x, y, regressors = regressorsx, color = 'r', #ci = None,
                         labels = False,lstyle='--', lw=0.7,  #annotate = (30,0.065),
                         fit_reg=1, scatter=1, plot_marg = 1,dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_Age_ChiAll_p.png'
                        )
plot_partial_correlation(df, x, y, regressors = regressorsx, color = 'g', 
                         labels = False,   #annotate = (25,0.06),
                         fit_reg=True, scatter=0, plot_marg = 0, dpi = dpi, xlim=xlim,ylim=ylim,
                         fig_name = 'CORR_Age_ChiAll_all.png'
                        )

In [ ]:
import matplotlib.font_manager
from IPython.core.display import HTML

def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

HTML("<div style='column-count: 2;'>{}</div>".format(code))

In [ ]:
font.family

In [ ]:
dfc